# <center>New Unseen Data Prediction Regression</center>
---

<center>Al Azhary Putera Satria</center>

In [1]:
# Run locally & on the cloud
import pandas as pd
import numpy as np
import pickle
from xgboost import XGBRegressor

In [11]:
new_data_point = pd.DataFrame({
    'Type': ['Sedan'],
    'Region': ['North America'],
    'Make': ['Toyota'],
    'Gear_Type': ['Automatic'],
    'Origin': ['Japan'],
    'Options': ['Basic'],
    'Year': [2022],
    'Engine_Size': [2.0],
    'Mileage': [15000],
    'Negotiable': [True],
    'Country_make': ['Japan'],
    'Condition': ['normally used'],
})

new_data_point

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Country_make,Condition
0,Sedan,North America,Toyota,Automatic,Japan,Basic,2022,2.0,15000,True,Japan,normally used


# Variable Need for Google Cloud (or You could just upload all of them to Github)

In [2]:
#import google cloud library
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [3]:
## set up authentication using services account 
import os
# Authenticate using service account
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "sa-development.json"
# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "../vertex-ai-ml/dti-ds-31329ac0651d.json"

project_id = 'dti-ds'
dataset_id = 'azhary_dataset_018'
table_id = 'batch_test'
region = 'us-central1'
bucket_name = 'azhary_gcs_018'
blob_name = 'model/batch_test.csv'

model_name = 'model_reg_Arab_Saudi_Usedcars.pkl'

## Google Cloud Storage

### Upload to Google Cloud Storage - skipped

### Retrieve from Google Cloud Storage

In [4]:
# Retrieve the model from Google Cloud Storage
try : 
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name) # Add bucket name
    blob_model = bucket.blob(f'model/{model_name}')
    blob_model.download_to_filename('model_reg_Arab_Saudi_Usedcars.pkl')

    print ("Read model succeeded")
except:
    raise TypeError("An exception occurred")

Read model succeeded


In [ ]:
# # Retrieve the dataset from Google Cloud Storage
# try : 
#     storage_client = storage.Client(project=project_id)
#     bucket = storage_client.get_bucket(bucket_name) # Add bucket name
#     blob_model = bucket.blob(blob_name)
#     blob_model.download_to_filename('car_insurance_real.csv')

#     print ("Read Data succeeded")
# except:
#     raise TypeError("An exception occurred")

## Bigquery

### Load from Bigquery

In [5]:
#load data from BQ
from google.cloud import bigquery
## using bigquery client 
# client = bigquery.Client(credentials=credentials,project=project_id)
client = bigquery.Client(project=project_id)

# query 
query_job = client.query(f"""select * from {dataset_id}.{table_id}""")
batch_test = query_job.result().to_dataframe()

# # df = df.loc[:, new_test_data.columns]
# auto_cloud.columns = new_data_point.columns
# #cleansing 
# # result = df.drop(['ID'], axis = 1)
# auto_cloud

/home/azharyputeras/miniconda3/envs/arab_reg_env/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [6]:
batch_test.head()

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Country_make,Condition
0,RX8,Jeddah,MG,Automatic,Saudi,Semi Full,2020,2.0,19000,False,UK,seldomly used
1,X,Riyadh,BMW,Automatic,Other,Full,2011,3.0,290,False,Germany,seldomly used
2,X,Riyadh,BMW,Automatic,Saudi,Full,2015,3.0,98000,False,Germany,normally used
3,X,Dammam,BMW,Automatic,Gulf Arabic,Semi Full,2017,3.0,71000,False,Germany,normally used
4,X,Riyadh,BMW,Automatic,Gulf Arabic,Full,2016,3.0,130000,False,Germany,overly used


# Model Prediction

## Cloud Data Prediction

In [16]:
pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 562.9 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 34.7 MB/s eta 0:00:0000:010:01m
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Load the model
with open('model_reg_Arab_Saudi_Usedcars.pkl', 'rb') as f:
    loaded_model = pickle.load(f)



In [8]:
cheap_car = pd.DataFrame({
    'Type': ['Grand Max'],
    'Region': ['Sabya'],
    'Make': ['Daihatsu'],
    'Gear_Type': ['Manual'],
    'Origin': ['Other'],
    'Options': ['Semi Full'],
    'Year': [2008],
    'Engine_Size': [4.4],
    'Mileage': [225000],
    'Negotiable': [True],
    'Country_make': ['Japan'],
    'Condition': ['normally used']
})


y_pred_cloud_new_data = loaded_model.predict(cheap_car)
y_pred_cloud_new_data_origin = np.expm1(y_pred_cloud_new_data)
y_pred_cloud_new_data_origin

array([43490.082], dtype=float32)

In [9]:
expensive_car = pd.DataFrame({
    'Type': ['Bentayga'],
    'Region': ['Sakaka'],
    'Make': ['Bentley'],
    'Gear_Type': ['Automatic'],
    'Origin': ['Saudi'],
    'Options': ['Full'],
    'Year': [2016],
    'Engine_Size': [3.6],
    'Mileage': [10000],
    'Negotiable': [True],
    'Country_make': ['UK'],
    'Condition': ['seldomly used']
})


y_pred_cloud_new_data = loaded_model.predict(expensive_car)
y_pred_cloud_new_data_origin = np.expm1(y_pred_cloud_new_data)
y_pred_cloud_new_data_origin

array([205550.88], dtype=float32)

In [10]:
# Load the model
with open('model_reg_Arab_Saudi_Usedcars.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

y_pred = loaded_model.predict(batch_test)
y_pred_origin = np.expm1(y_pred)
y_pred_origin.shape

(903,)

In [11]:
batch_test.head()

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Country_make,Condition
0,RX8,Jeddah,MG,Automatic,Saudi,Semi Full,2020,2.0,19000,False,UK,seldomly used
1,X,Riyadh,BMW,Automatic,Other,Full,2011,3.0,290,False,Germany,seldomly used
2,X,Riyadh,BMW,Automatic,Saudi,Full,2015,3.0,98000,False,Germany,normally used
3,X,Dammam,BMW,Automatic,Gulf Arabic,Semi Full,2017,3.0,71000,False,Germany,normally used
4,X,Riyadh,BMW,Automatic,Gulf Arabic,Full,2016,3.0,130000,False,Germany,overly used


In [12]:
batch_test['price_pred']=y_pred_origin
batch_test.head()

,Type,Region,Make,Gear_Type,Origin,Options,Year,Engine_Size,Mileage,Negotiable,Country_make,Condition,price_pred
0,RX8,Jeddah,MG,Automatic,Saudi,Semi Full,2020,2.0,19000,False,UK,seldomly used,88084.781250
1,X,Riyadh,BMW,Automatic,Other,Full,2011,3.0,290,False,Germany,seldomly used,62601.019531
2,X,Riyadh,BMW,Automatic,Saudi,Full,2015,3.0,98000,False,Germany,normally used,125612.125000
3,X,Dammam,BMW,Automatic,Gulf Arabic,Semi Full,2017,3.0,71000,False,Germany,normally used,174006.968750
4,X,Riyadh,BMW,Automatic,Gulf Arabic,Full,2016,3.0,130000,False,Germany,overly used,144210.187500


In [13]:
from google.cloud import bigquery

table_id = 'batch_test_predicted2'

# Construct a bigquery client object
client = bigquery.Client()

# define the full table id
table_full_id = f"{client.project}.{dataset_id}.{table_id}"

batch_test.columns = ['_'.join(i.split(' ')) for i in batch_test.columns]

job = client.load_table_from_dataframe(batch_test,table_full_id)

job.result()
print(f"Loaded {job.output_rows} rows into {table_full_id}")

Loaded 903 rows into dti-ds.azhary_dataset_018.batch_test_predicted2


In [26]:
# # loaded_model.predict(X_test.iloc[3:13])
# y_pred_file_cloud = loaded_model.predict(auto_cloud)
# y_pred_file_cloud[:13]

array([ 5625.1167, 19218.45  ], dtype=float32)

In [ ]:
# auto_cloud['CLV Prediction'] = y_pred_file_cloud
# auto_cloud

---
<center>Thank You</center>

---